# COVID19 severe (hospitalized) cases in Brazil: ML logistic regression

Here, publicly available data for hospitalized cases in Brazil is used to perform a retrospective cross-sectional observational study.

It means that I want to infer the paramaters driving the outcome from data gathered from hospitals.
The data has been downloaded, selected and preprocessed before the steps described in this notebook.

The aim is to compare different variants and infer what parameters are driving the severity and the outcome. This will provide answers to the following questions:
<ol>
    <li> Can we predict the outcome?</li>
    <li> If yes, what are the parameters influencing the outcome?</li>
</ol>
<br>
I study only the main outcome: cured/death.

This is applied to four different periods of time when four different variants where dominant (>=80% of samples analyzed were corresponding to the variant of interest, source: GISAID database):
- Delta
- Omicron BA.1
- Omicron BA.2
- Omicron BA.4/BA.5

Specifically, in this notebook, **I show the results of Machine Learning logistic regression on the main outcome**.

> **Data source**: all the data has been taken from the Brazilian Ministry of Health https://opendatasus.saude.gov.br/organization/ministerio-da-saude
>
>It requires translation from Portuguese to English

> **<font size=5 color='red'>IMPORTANT DISCLAIMER</font>**: in this work, we extract data about **HOSPITALIZED** patients, by definition people already severely affected by the disease, hence a strongly biased sample unfit to get the whole picture about all the aspects of COVID19

In [1]:
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns

# Read data

In [2]:
comorb_list = ['cardiovascular_disease','hematologic_disease','down_syndrom','liver_disease','comorb_asthma','diabetes',
               'neurological_disease','chronic_lung_disease','weaken_immune_system','renal_disease','obesity','puerperal',
               'other_comorbidities']
symptom_list = ['abdominal_pain','altered_smell_taste','anxiety','asthma','asymptomatic','back_pain','blocked_nose',
                'blurred_vision','body_aches','chest_pain','chills_or_shivers','cyanosis','delirium','diarrhoea',
                'discomfort','dizzy_light_headed','eye_soreness','fatigue_weakness','fever','gastrointestinal_symptoms',
                'headache','insomnia','loss_of_appetite','low_o2_sat','myalgia','nausea','other','other_respiratory',
                'other_skin','persistent_cough','retro_orbital_pain','runny_nose','seizures','shortness_of_breath',
                'skin_lesions','sneezing','sore_throat','unusual_joint_pains']
variants_name = ['Delta','BA.1.X','BA.2.X','BA.4/5.X']

In [3]:
df_encoded = pd.read_parquet('df_ML.pq')

In [4]:
state_list = df_encoded.columns[df_encoded.columns.str.contains('state')].tolist()
variant_list = df_encoded.columns[df_encoded.columns.str.contains('variant')].tolist()
ethnicity_list = df_encoded.columns[df_encoded.columns.str.contains('ethnicity')].tolist()

# Machine Learning: Multivariate Logistic Regression

## Features selection for model

I exclude icu admission and invasive ventilation from parameters since they are strongly correlated with the final outcome (as logically expected)

I drop also noninvasive ventilation and length of stay: features created while hospitalized

I work on reducing the number of parameters going into the training.

In [5]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFECV
from statsmodels.stats.outliers_influence import variance_inflation_factor

I arbitrarly set the minimum number of parameters to go into the fitting to 15.

In [ ]:
list_features = []
for variant in variants_name:
    print(variant)
    df_variant = df_encoded[df_encoded['variant_'+variant]==1].drop(columns=['age_group','ventilation_invasive','icu_adm','pregnancy','puerperal','ventilation_noninvasive','length_stay']).drop(columns=variant_list)
    df_variant_filled = df_variant.dropna(axis=0)
    X, y = df_variant_filled.drop(['outcome'],1), df_variant_filled.outcome
    scaler = MinMaxScaler()
    X[['age','nb_comorbidities','delay_lastdose_onset','length_delay','nb_vaccine_dose']] = scaler.fit_transform(X[['age','nb_comorbidities','delay_lastdose_onset','length_delay','nb_vaccine_dose']])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
    logreg = LogisticRegression(solver='newton-cholesky')
    selector = RFECV(logreg,step=1,min_features_to_select=15)
    check_convergence = 0
    init_df = 0
    while (check_convergence==0):
        if init_df == 0:
            X_loop = X_train.copy()
            init_df = 1
        else:
            X_loop = X_train[feature_order[feature_order.ranking==1].feature]
        selector = selector.fit(X_loop,y_train)
        score = selector.score(X_test[X_loop.columns],y_test)
        print("Test score:{:.3f}".format(score),' ',end="")
        feature_order = pd.DataFrame()
        feature_order['feature'] = X_loop.columns
        feature_order['selected'] = selector.support_
        feature_order['ranking'] = selector.ranking_
        print('nb of features:',len(X_loop.columns))
        if len(X_loop.columns) == len(X_train[feature_order[feature_order.ranking==1].feature].columns):
            check_convergence = 1
            list_features.append(X_train[feature_order[feature_order.ranking==1].feature].columns.tolist())

Delta
Test score:0.709  nb of features: 89
Test score:0.708  nb of features: 51
Test score:0.702  nb of features: 47
Test score:0.703  nb of features: 39
Test score:0.703  nb of features: 38
BA.1.X
Test score:0.703  nb of features: 89
Test score:0.703  nb of features: 55
BA.2.X
Test score:0.765  nb of features: 89
Test score:0.763  nb of features: 17
Test score:0.763  nb of features: 15
BA.4/5.X
Test score:0.732  nb of features: 89
Test score:0.732  nb of features: 79
Test score:0.732  nb of features: 75
Test score:0.734  nb of features: 71
Test score:0.735  nb of features: 64
Test score:0.735  nb of features: 50
Test score:0.732  nb of features: 48


## Fit

In [ ]:
list_param_toscale = ['age','nb_comorbidities','delay_lastdose_onset','length_delay','nb_vaccine_dose']
for variant in variants_name:
    print(variant)
    df_variant = df_encoded[df_encoded['variant_'+variant]==1].drop(columns=['age_group','ventilation_invasive','icu_adm','pregnancy','puerperal','ventilation_noninvasive','length_stay']).drop(columns=variant_list)
    param_variant = list_features[variants_name.index(variant)].copy()
    param_variant.append('outcome')
    df_variant = df_variant[param_variant]
    df_variant_filled = df_variant.dropna(axis=0)
    X, y = df_variant_filled.drop(['outcome'],1), df_variant_filled.outcome
    param_toscale = []
    for param in list_param_toscale:
        if param in df_variant_filled.columns:
                param_toscale.append(param)
    if len(param_toscale)>0:
        scaler = MinMaxScaler()
        X[param_toscale] = scaler.fit_transform(X[param_toscale])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)
    logm1 = sm.GLM(y_train,(sm.add_constant(X_train)),family = sm.families.Binomial())
#    logreg = LogisticRegression(solver='newton-cholesky')
    if variant == 'Delta':
        res_delta = logm1.fit()
    elif variant == 'BA.1.X':
        res_ba1x = logm1.fit()
    elif variant == 'BA.2.X':
        res_ba2x = logm1.fit()
    elif variant == 'BA.4/5.X':
        res_ba45x = logm1.fit()

## Model evaluation

### Delta

In [ ]:
for variant in ['Delta']:
    print(variant)
    df_variant = df_encoded[df_encoded['variant_'+variant]==1].drop(columns=['age_group','ventilation_invasive','icu_adm','pregnancy','puerperal','ventilation_noninvasive','length_stay']).drop(columns=variant_list)
    param_variant = list_features[variants_name.index(variant)].copy()
    param_variant.append('outcome')
    df_variant = df_variant[param_variant]
    df_variant_filled = df_variant.dropna(axis=0)
    X, y = df_variant_filled.drop(['outcome'],1), df_variant_filled.outcome
    param_toscale = []
    for param in list_param_toscale:
        if param in df_variant_filled.columns:
                param_toscale.append(param)
    if len(param_toscale)>0:
        scaler = MinMaxScaler()
        X[param_toscale] = scaler.fit_transform(X[param_toscale])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

#### Compare predicted outcome with observed outcome

In [ ]:
y_train_pred = res_delta.predict(sm.add_constant(X_train))
y_train_pred = y_train_pred.values.reshape(-1)
y_train_pred_final = pd.DataFrame({'Converted':y_train.values, 'Conversion_Prob':y_train_pred})
y_train_pred_final['Predicted'] = y_train_pred_final.Conversion_Prob.map(lambda x: 1 if x>0.5 else 0)
y_train_pred_final.head()

#### Confusion matrix

Visualize algorithm performance


|True Negative | False Positive|
|-|-|
|**False Negative** | **True Positive**|

In [ ]:
#confusion matrix

from sklearn import metrics

confusion = metrics.confusion_matrix(y_train_pred_final.Converted, y_train_pred_final.Predicted)
print(confusion)

In [ ]:
import seaborn as sns

df_cm = pd.DataFrame(confusion,index=['Cured (obs)','Dead (obs)'],columns=['Cured (pred)','Dead (pred)'])
sns.heatmap(df_cm,annot=True,fmt='4d',cmap='BrBG')

#### Accuracy

(TP+TN) / (TP+TN+FP+FN)

In [ ]:
print(metrics.accuracy_score(y_train_pred_final.Converted,y_train_pred_final.Predicted))

In [ ]:
TP = confusion[1,1]
TN = confusion[0,0]
FP = confusion[0,1]
FN = confusion[1,0]

In [ ]:
#precision
print('Precision:',TP/(TP+FP))
#recall
print('Recall:',TP/(TP+FN))
#f-score
print('f-score:',((TP/(TP+FP))*(TP/(TP+FN)))/((TP/(TP+FP))+(TP/(TP+FN))))

As the algorithm is set, it predicts the outcome for the training set correcly 70% of the time

But the is data unbalanced: 66% of cured vs. 34% of dead in the training data.

Model predicts correctly death in only **36%** of the cases (recall).

#### Model optimization
#### ROC function

In [ ]:
# ROC function

def draw_roc( actual, probs ):
    fpr, tpr, thresholds = metrics.roc_curve( actual, probs,
                                              drop_intermediate = False )
    auc_score = metrics.roc_auc_score( actual, probs )
    plt.figure(figsize=(5, 5))
    plt.plot( fpr, tpr, label='ROC curve (area = %0.2f)' % auc_score )
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate or [1 - True Negative Rate]')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()

    return None

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve( y_train_pred_final.Converted, y_train_pred_final.Conversion_Prob, drop_intermediate = False )


In [ ]:
draw_roc(y_train_pred_final.Converted, y_train_pred_final.Conversion_Prob)

#### trade off: accuracy, specificity, sensitivity

In [ ]:
numbers = [float(x)/10 for x in range(10)]
numbers = np.linspace(0,1,21)
for i in numbers:
    y_train_pred_final['%.2f'%i]= y_train_pred_final.Conversion_Prob.map(lambda x: 1 if x > i else 0)
y_train_pred_final.head()

In [ ]:
cutoff_df = pd.DataFrame( columns = ['prob','accuracy','sensi','speci'])
from sklearn.metrics import confusion_matrix

num = np.linspace(0,1,21)
for i in num:
    cm1 = metrics.confusion_matrix(y_train_pred_final.Converted, y_train_pred_final['%.2f'%i] )
    total1=sum(sum(cm1))
    accuracy = (cm1[0,0]+cm1[1,1])/total1
    
    speci = cm1[0,0]/(cm1[0,0]+cm1[0,1])
    sensi = cm1[1,1]/(cm1[1,0]+cm1[1,1])
    cutoff_df.loc[i] =[ i ,accuracy,sensi,speci]
print(cutoff_df)

In [ ]:
cutoff_df.plot.line(x='prob', y=['accuracy','sensi','speci'])

In [ ]:
y_train_pred_final['final_predicted'] = y_train_pred_final.Conversion_Prob.map( lambda x: 1 if x > 0.35 else 0)
y_train_pred_final.head()

#### Evaluation model with new cut off

In [ ]:
metrics.accuracy_score(y_train_pred_final.Converted, y_train_pred_final.final_predicted)

In [ ]:
confusion2 = metrics.confusion_matrix(y_train_pred_final.Converted, y_train_pred_final.final_predicted )
print(confusion2)
df_cm = pd.DataFrame(confusion2,index=['Cured (obs)','Dead (obs)'],columns=['Cured (pred)','Dead (pred)'])
sns.heatmap(df_cm,annot=True,fmt='4d',cmap='BrBG')

In [ ]:
from sklearn.metrics import precision_recall_curve
p, r, thresholds = precision_recall_curve(y_train_pred_final.Converted, y_train_pred_final.Conversion_Prob)
plt.plot(thresholds, p[:-1], "b:")
plt.plot(thresholds, r[:-1], "r--")

cut off value near value where precision and recall intersect: keep model as it is

## Model prediction on test set

In [ ]:
y_test_pred = res_delta.predict(sm.add_constant(X_test))
y_pred_1 = pd.DataFrame(y_test_pred)
y_test_df = pd.DataFrame(y_test)
y_pred_1.reset_index(drop=True, inplace=True)
y_test_df.reset_index(drop=True, inplace=True)
y_pred_final = pd.concat([y_test_df, y_pred_1],axis=1)
y_pred_final= y_pred_final.rename(columns = {0 : 'Conversion_Prob'})
y_pred_final['final_predicted'] = y_pred_final.Conversion_Prob.map(lambda x: 1 if x > 0.35 else 0)
print(metrics.accuracy_score(y_pred_final['outcome'], y_pred_final.final_predicted))
confusion3 = metrics.confusion_matrix(y_pred_final['outcome'], y_pred_final.final_predicted )
print(confusion3)
df_cm = pd.DataFrame(confusion3,index=['Cured (obs)','Dead (obs)'],columns=['Cured (pred)','Dead (pred)'])
sns.heatmap(df_cm,annot=True,fmt='4d',cmap='BrBG')
TP = confusion3[1,1] 
TN = confusion3[0,0] 
FP = confusion3[0,1] 
FN = confusion3[1,0] 
#precision
print('Precision:',TP/(TP+FP))
#recall
print('Recall:',TP/(TP+FN))
#f-score
print('f-score:',((TP/(TP+FP))*(TP/(TP+FN)))/((TP/(TP+FP))+(TP/(TP+FN))))

# BA.1.X

## Fit and model evaluation

In [ ]:
for variant in ['BA.1.X']:
    df_variant = df_encoded[df_encoded['variant_'+variant]==1].drop(columns=['age_group','ventilation_invasive','icu_adm','pregnancy','puerperal','ventilation_noninvasive','length_stay']).drop(columns=variant_list)
    param_variant = list_features[variants_name.index(variant)].copy()
    param_variant.append('outcome')
    df_variant = df_variant[param_variant]
    df_variant_filled = df_variant.dropna(axis=0)
    X, y = df_variant_filled.drop(['outcome'],1), df_variant_filled.outcome
    param_toscale = []
    for param in list_param_toscale:
        if param in df_variant_filled.columns:
                param_toscale.append(param)
    if len(param_toscale)>0:
        scaler = MinMaxScaler()
        X[param_toscale] = scaler.fit_transform(X[param_toscale])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)
y_train_pred = res_ba1x.predict(sm.add_constant(X_train))
y_train_pred = y_train_pred.values.reshape(-1)
y_train_pred_final = pd.DataFrame({'Converted':y_train.values, 'Conversion_Prob':y_train_pred})
y_train_pred_final['Predicted'] = y_train_pred_final.Conversion_Prob.map(lambda x: 1 if x>0.5 else 0)
#confusion matrix
confusion = metrics.confusion_matrix(y_train_pred_final.Converted, y_train_pred_final.Predicted)
print(confusion)
print(metrics.accuracy_score(y_train_pred_final.Converted,y_train_pred_final.Predicted))
TP = confusion[1,1]
TN = confusion[0,0]
FP = confusion[0,1]
FN = confusion[1,0]
#precision
print('Precision:',TP/(TP+FP))
#recall
print('Recall:',TP/(TP+FN))
#f-score
print('f-score:',((TP/(TP+FP))*(TP/(TP+FN)))/((TP/(TP+FP))+(TP/(TP+FN))))
fpr, tpr, thresholds = metrics.roc_curve( y_train_pred_final.Converted, y_train_pred_final.Conversion_Prob, drop_intermediate = False )
draw_roc(y_train_pred_final.Converted, y_train_pred_final.Conversion_Prob)
numbers = [float(x)/10 for x in range(10)]
numbers = np.linspace(0,1,21)
for i in numbers:
    y_train_pred_final['%.2f'%i]= y_train_pred_final.Conversion_Prob.map(lambda x: 1 if x > i else 0)
cutoff_df = pd.DataFrame( columns = ['prob','accuracy','sensi','speci'])
num = np.linspace(0,1,21)
for i in num:
    cm1 = metrics.confusion_matrix(y_train_pred_final.Converted, y_train_pred_final['%.2f'%i] )
    total1=sum(sum(cm1))
    accuracy = (cm1[0,0]+cm1[1,1])/total1
    
    speci = cm1[0,0]/(cm1[0,0]+cm1[0,1])
    sensi = cm1[1,1]/(cm1[1,0]+cm1[1,1])
    cutoff_df.loc[i] =[ i ,accuracy,sensi,speci]
print(cutoff_df)
cutoff_df.plot.line(x='prob', y=['accuracy','sensi','speci'])

In [ ]:
y_train_pred_final['final_predicted'] = y_train_pred_final.Conversion_Prob.map( lambda x: 1 if x > 0.35 else 0)
metrics.accuracy_score(y_train_pred_final.Converted, y_train_pred_final.final_predicted)
confusion2 = metrics.confusion_matrix(y_train_pred_final.Converted, y_train_pred_final.final_predicted )
print(confusion2)
df_cm = pd.DataFrame(confusion2,index=['Cured (obs)','Dead (obs)'],columns=['Cured (pred)','Dead (pred)'])
sns.heatmap(df_cm,annot=True,fmt='4d',cmap='BrBG')

In [ ]:
p, r, thresholds = precision_recall_curve(y_train_pred_final.Converted, y_train_pred_final.Conversion_Prob)
plt.plot(thresholds, p[:-1], "b:")
plt.plot(thresholds, r[:-1], "r--")

## Model prediction on test set

In [ ]:
y_test_pred = res_ba1x.predict(sm.add_constant(X_test))
y_pred_1 = pd.DataFrame(y_test_pred)
y_test_df = pd.DataFrame(y_test)
y_pred_1.reset_index(drop=True, inplace=True)
y_test_df.reset_index(drop=True, inplace=True)
y_pred_final = pd.concat([y_test_df, y_pred_1],axis=1)
y_pred_final= y_pred_final.rename(columns = {0 : 'Conversion_Prob'})
y_pred_final['final_predicted'] = y_pred_final.Conversion_Prob.map(lambda x: 1 if x > 0.35 else 0)
print(metrics.accuracy_score(y_pred_final['outcome'], y_pred_final.final_predicted))
confusion3 = metrics.confusion_matrix(y_pred_final['outcome'], y_pred_final.final_predicted )
print(confusion3)
df_cm = pd.DataFrame(confusion3,index=['Cured (obs)','Dead (obs)'],columns=['Cured (pred)','Dead (pred)'])
sns.heatmap(df_cm,annot=True,fmt='4d',cmap='BrBG')
TP = confusion3[1,1] 
TN = confusion3[0,0] 
FP = confusion3[0,1] 
FN = confusion3[1,0] 
#precision
print('Precision:',TP/(TP+FP))
#recall
print('Recall:',TP/(TP+FN))
#f-score
print('f-score:',((TP/(TP+FP))*(TP/(TP+FN)))/((TP/(TP+FP))+(TP/(TP+FN))))

# BA.2.X

## Fit and model evaluation

In [ ]:
for variant in ['BA.2.X']:
    df_variant = df_encoded[df_encoded['variant_'+variant]==1].drop(columns=['age_group','ventilation_invasive','icu_adm','pregnancy','puerperal','ventilation_noninvasive','length_stay']).drop(columns=variant_list)
    param_variant = list_features[variants_name.index(variant)].copy()
    param_variant.append('outcome')
    df_variant = df_variant[param_variant]
    df_variant_filled = df_variant.dropna(axis=0)
    X, y = df_variant_filled.drop(['outcome'],1), df_variant_filled.outcome
    param_toscale = []
    for param in list_param_toscale:
        if param in df_variant_filled.columns:
                param_toscale.append(param)
    if len(param_toscale)>0:
        scaler = MinMaxScaler()
        X[param_toscale] = scaler.fit_transform(X[param_toscale])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)
y_train_pred = res_ba2x.predict(sm.add_constant(X_train))
y_train_pred = y_train_pred.values.reshape(-1)
y_train_pred_final = pd.DataFrame({'Converted':y_train.values, 'Conversion_Prob':y_train_pred})
y_train_pred_final['Predicted'] = y_train_pred_final.Conversion_Prob.map(lambda x: 1 if x>0.5 else 0)
#confusion matrix
confusion = metrics.confusion_matrix(y_train_pred_final.Converted, y_train_pred_final.Predicted)
print(confusion)
print(metrics.accuracy_score(y_train_pred_final.Converted,y_train_pred_final.Predicted))
TP = confusion[1,1]
TN = confusion[0,0]
FP = confusion[0,1]
FN = confusion[1,0]
#precision
print('Precision:',TP/(TP+FP))
#recall
print('Recall:',TP/(TP+FN))
#f-score
print('f-score:',((TP/(TP+FP))*(TP/(TP+FN)))/((TP/(TP+FP))+(TP/(TP+FN))))
fpr, tpr, thresholds = metrics.roc_curve( y_train_pred_final.Converted, y_train_pred_final.Conversion_Prob, drop_intermediate = False )
draw_roc(y_train_pred_final.Converted, y_train_pred_final.Conversion_Prob)
numbers = [float(x)/10 for x in range(10)]
numbers = np.linspace(0,1,21)
for i in numbers:
    y_train_pred_final['%.2f'%i]= y_train_pred_final.Conversion_Prob.map(lambda x: 1 if x > i else 0)
cutoff_df = pd.DataFrame( columns = ['prob','accuracy','sensi','speci'])
num = np.linspace(0,1,21)
for i in num:
    cm1 = metrics.confusion_matrix(y_train_pred_final.Converted, y_train_pred_final['%.2f'%i] )
    total1=sum(sum(cm1))
    accuracy = (cm1[0,0]+cm1[1,1])/total1
    
    speci = cm1[0,0]/(cm1[0,0]+cm1[0,1])
    sensi = cm1[1,1]/(cm1[1,0]+cm1[1,1])
    cutoff_df.loc[i] =[ i ,accuracy,sensi,speci]
print(cutoff_df)
cutoff_df.plot.line(x='prob', y=['accuracy','sensi','speci'])

In [ ]:
y_train_pred_final['final_predicted'] = y_train_pred_final.Conversion_Prob.map( lambda x: 1 if x > 0.25 else 0)
metrics.accuracy_score(y_train_pred_final.Converted, y_train_pred_final.final_predicted)
confusion2 = metrics.confusion_matrix(y_train_pred_final.Converted, y_train_pred_final.final_predicted )
print(confusion2)
df_cm = pd.DataFrame(confusion2,index=['Cured (obs)','Dead (obs)'],columns=['Cured (pred)','Dead (pred)'])
sns.heatmap(df_cm,annot=True,fmt='4d',cmap='BrBG')

In [ ]:
p, r, thresholds = precision_recall_curve(y_train_pred_final.Converted, y_train_pred_final.Conversion_Prob)
plt.plot(thresholds, p[:-1], "b:")
plt.plot(thresholds, r[:-1], "r--")

## Model prediction on test set

In [ ]:
y_test_pred = res_ba2x.predict(sm.add_constant(X_test))
y_pred_1 = pd.DataFrame(y_test_pred)
y_test_df = pd.DataFrame(y_test)
y_pred_1.reset_index(drop=True, inplace=True)
y_test_df.reset_index(drop=True, inplace=True)
y_pred_final = pd.concat([y_test_df, y_pred_1],axis=1)
y_pred_final= y_pred_final.rename(columns = {0 : 'Conversion_Prob'})
y_pred_final['final_predicted'] = y_pred_final.Conversion_Prob.map(lambda x: 1 if x > 0.25 else 0)
print(metrics.accuracy_score(y_pred_final['outcome'], y_pred_final.final_predicted))
confusion3 = metrics.confusion_matrix(y_pred_final['outcome'], y_pred_final.final_predicted )
print(confusion3)
df_cm = pd.DataFrame(confusion3,index=['Cured (obs)','Dead (obs)'],columns=['Cured (pred)','Dead (pred)'])
sns.heatmap(df_cm,annot=True,fmt='4d',cmap='BrBG')
TP = confusion3[1,1] 
TN = confusion3[0,0] 
FP = confusion3[0,1] 
FN = confusion3[1,0] 
#precision
print('Precision:',TP/(TP+FP))
#recall
print('Recall:',TP/(TP+FN))
#f-score
print('f-score:',((TP/(TP+FP))*(TP/(TP+FN)))/((TP/(TP+FP))+(TP/(TP+FN))))

# BA.4/5.X

## Fit and model evaluation

In [ ]:
for variant in ['BA.4/5.X']:
    df_variant = df_encoded[df_encoded['variant_'+variant]==1].drop(columns=['age_group','ventilation_invasive','icu_adm','pregnancy','puerperal','ventilation_noninvasive','length_stay']).drop(columns=variant_list)
    param_variant = list_features[variants_name.index(variant)].copy()
    param_variant.append('outcome')
    df_variant = df_variant[param_variant]
    df_variant_filled = df_variant.dropna(axis=0)
    X, y = df_variant_filled.drop(['outcome'],1), df_variant_filled.outcome
    param_toscale = []
    for param in list_param_toscale:
        if param in df_variant_filled.columns:
                param_toscale.append(param)
    if len(param_toscale)>0:
        scaler = MinMaxScaler()
        X[param_toscale] = scaler.fit_transform(X[param_toscale])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)
y_train_pred = res_ba45x.predict(sm.add_constant(X_train))
y_train_pred = y_train_pred.values.reshape(-1)
y_train_pred_final = pd.DataFrame({'Converted':y_train.values, 'Conversion_Prob':y_train_pred})
y_train_pred_final['Predicted'] = y_train_pred_final.Conversion_Prob.map(lambda x: 1 if x>0.5 else 0)
#confusion matrix
confusion = metrics.confusion_matrix(y_train_pred_final.Converted, y_train_pred_final.Predicted)
print(confusion)
print(metrics.accuracy_score(y_train_pred_final.Converted,y_train_pred_final.Predicted))
TP = confusion[1,1]
TN = confusion[0,0]
FP = confusion[0,1]
FN = confusion[1,0]
#precision
print('Precision:',TP/(TP+FP))
#recall
print('Recall:',TP/(TP+FN))
#f-score
print('f-score:',((TP/(TP+FP))*(TP/(TP+FN)))/((TP/(TP+FP))+(TP/(TP+FN))))
fpr, tpr, thresholds = metrics.roc_curve( y_train_pred_final.Converted, y_train_pred_final.Conversion_Prob, drop_intermediate = False )
draw_roc(y_train_pred_final.Converted, y_train_pred_final.Conversion_Prob)
numbers = [float(x)/10 for x in range(10)]
numbers = np.linspace(0,1,21)
for i in numbers:
    y_train_pred_final['%.2f'%i]= y_train_pred_final.Conversion_Prob.map(lambda x: 1 if x > i else 0)
cutoff_df = pd.DataFrame( columns = ['prob','accuracy','sensi','speci'])
num = np.linspace(0,1,21)
for i in num:
    cm1 = metrics.confusion_matrix(y_train_pred_final.Converted, y_train_pred_final['%.2f'%i] )
    total1=sum(sum(cm1))
    accuracy = (cm1[0,0]+cm1[1,1])/total1
    
    speci = cm1[0,0]/(cm1[0,0]+cm1[0,1])
    sensi = cm1[1,1]/(cm1[1,0]+cm1[1,1])
    cutoff_df.loc[i] =[ i ,accuracy,sensi,speci]
print(cutoff_df)
cutoff_df.plot.line(x='prob', y=['accuracy','sensi','speci'])

In [ ]:
y_train_pred_final['final_predicted'] = y_train_pred_final.Conversion_Prob.map( lambda x: 1 if x > 0.275 else 0)
metrics.accuracy_score(y_train_pred_final.Converted, y_train_pred_final.final_predicted)
confusion2 = metrics.confusion_matrix(y_train_pred_final.Converted, y_train_pred_final.final_predicted )
print(confusion2)
df_cm = pd.DataFrame(confusion2,index=['Cured (obs)','Dead (obs)'],columns=['Cured (pred)','Dead (pred)'])
sns.heatmap(df_cm,annot=True,fmt='4d',cmap='BrBG')

In [ ]:
p, r, thresholds = precision_recall_curve(y_train_pred_final.Converted, y_train_pred_final.Conversion_Prob)
plt.plot(thresholds, p[:-1], "b:")
plt.plot(thresholds, r[:-1], "r--")

## Model prediction on test set

In [ ]:
y_test_pred = res_ba45x.predict(sm.add_constant(X_test))
y_pred_1 = pd.DataFrame(y_test_pred)
y_test_df = pd.DataFrame(y_test)
y_pred_1.reset_index(drop=True, inplace=True)
y_test_df.reset_index(drop=True, inplace=True)
y_pred_final = pd.concat([y_test_df, y_pred_1],axis=1)
y_pred_final= y_pred_final.rename(columns = {0 : 'Conversion_Prob'})
y_pred_final['final_predicted'] = y_pred_final.Conversion_Prob.map(lambda x: 1 if x > 0.25 else 0)
print(metrics.accuracy_score(y_pred_final['outcome'], y_pred_final.final_predicted))
confusion3 = metrics.confusion_matrix(y_pred_final['outcome'], y_pred_final.final_predicted )
print(confusion3)
df_cm = pd.DataFrame(confusion3,index=['Cured (obs)','Dead (obs)'],columns=['Cured (pred)','Dead (pred)'])
sns.heatmap(df_cm,annot=True,fmt='4d',cmap='BrBG')
TP = confusion3[1,1] 
TN = confusion3[0,0] 
FP = confusion3[0,1] 
FN = confusion3[1,0] 
#precision
print('Precision:',TP/(TP+FP))
#recall
print('Recall:',TP/(TP+FN))
#f-score
print('f-score:',((TP/(TP+FP))*(TP/(TP+FN)))/((TP/(TP+FP))+(TP/(TP+FN))))